In [27]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sn

In [28]:
da = pd.read_csv('nhanes.csv')

In [29]:
da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


INVESTIGATING AND CLEANING DATA

In [30]:
da['SMQ020x'] = da.SMQ020.replace({1:'Yes', 2:'No', 7:np.nan, 9:np.nan})

In [31]:
da.SMQ020x

0       Yes
1       Yes
2       Yes
3        No
4        No
       ... 
5730    Yes
5731     No
5732    Yes
5733    Yes
5734     No
Name: SMQ020x, Length: 5735, dtype: object

In [32]:
da['RIAGENDRx'] = da.RIAGENDR.replace({1:'Male',2:'Female'})

In [33]:
da.RIAGENDRx

0         Male
1         Male
2         Male
3       Female
4       Female
         ...  
5730    Female
5731      Male
5732    Female
5733      Male
5734    Female
Name: RIAGENDRx, Length: 5735, dtype: object

In [34]:
dx = da[["SMQ020x", "RIAGENDRx"]].dropna()

In [35]:
pd.crosstab(dx.SMQ020x, dx.RIAGENDRx)

RIAGENDRx,Female,Male
SMQ020x,,
No,2066,1340
Yes,906,1413


In [36]:
dx["SMQ020x"] = dx.SMQ020x.replace({"Yes":1,"No":0})
dx

,SMQ020x,RIAGENDRx
0,1,Male
1,1,Male
2,1,Male
3,0,Female
4,0,Female
...,...,...
5730,1,Female
5731,0,Male
5732,1,Female
5733,1,Male


In [37]:
dx = dx.groupby("RIAGENDRx").agg({"SMQ020x":[np.mean, np.size]})
dx

SMQ020x      
               mean  size
RIAGENDRx                
Female     0.304845  2972
Male       0.513258  2753

Difference in Two Population Proportions

In [44]:
p = 0.304845
n = 2972
se_female = np.sqrt(p * (1 - p)/n)
se_female2 = p * (1 - p)/n
se_female, se_female2

(0.00844415041930423, 7.130367630383579e-05)

In [45]:
p = 0.513258
n = 2753
se_male = np.sqrt(p * (1 - p)/n)
se_male2 = p * (1 - p)/n
se_male, se_male2

(0.009526078787008965, 9.074617705630221e-05)

In [46]:
se_diff = np.sqrt(se_female**2 + se_male**2)
se_diff2 = np.sqrt(se_female2 + se_male2)
se_diff, se_diff2

(0.012729880335656654, 0.012729880335656656)

In [43]:
d = 0.304845 - 0.513258
lcb = d - 1.96 * se_diff
ucb = d + 1.96 *se_diff
(lcb,ucb)

(-0.23336356545788706, -0.18346243454211297)

Difference In Two Population Mean

In [47]:
da.BMXBMI.head()

0    27.8
1    30.8
2    28.8
3    42.4
4    20.3
Name: BMXBMI, dtype: float64

In [49]:
da.groupby("RIAGENDRx").agg({"BMXBMI": [np.mean,np.std, np.size]})

BMXBMI                  
                mean       std    size
RIAGENDRx                             
Female     29.939946  7.753319  2976.0
Male       28.778072  6.252568  2759.0

In [50]:
sen_female = 7.753319/np.sqrt(2976)
sen_male = 6.252568/np.sqrt(2759)
sen_female, sen_male

(0.14212523289878048, 0.11903716451870151)

In [53]:
sen_diff = np.sqrt(sen_female**2 + sen_male**2)
sen_diff

0.18538993598139303

In [54]:
mean_diff = 29.939946 - 28.778072

In [56]:
lcb = mean_diff - 1.96 * sen_diff
ucb = mean_diff + 1.96 * sen_diff
lcb,ucb

(0.798509725476467, 1.5252382745235278)